In [2]:
from IPython import get_ipython
ip = get_ipython()
ip.run_line_magic("reload_ext", "autoreload")  # these will enable module autoreloading
ip.run_line_magic("autoreload", "2")

### Load in Exports

In [8]:
import os
from lxml import etree
import src.BrokerageExportProviders.Brokerages.IBKR.IbkrBrokerageExportProvider as ibrk
import src.ConfigurationProvider.Configuration as conf
import src.Core.FinancialEvents.GroupingProcessor.StagingToUnderlyingProcessor as stup
import arrow

rootOfProject = os.getcwd()

reportsDirectory = os.path.join(rootOfProject, "imports")
generatedDirectory = os.path.join(rootOfProject, "exports")
configDirectory = os.path.join(rootOfProject, "config")

configProvider = conf.ConfigurationProvider(configDirectory)
taxPayerInfo = configProvider.getConfig()

ibkrProvider = ibrk.IbkrBrokerageExportProvider()
brokerExports = ibkrProvider.getListOfReportsAvailableForBroker(reportsDirectory)
print(brokerExports)



def loadFileAndExtractLines(file: str):  
    transactions = ibkrProvider.getBrokerEventsForReport(file)
    return transactions

brokerReports = list(map(loadFileAndExtractLines, brokerExports))
mergedReports = ibkrProvider.mergeBrokerEvents(brokerReports)

['/mnt/extra/Projects/IB-tax-calculator/imports/Tax_Relevant_Export_2023.xml', '/mnt/extra/Projects/IB-tax-calculator/imports/Tax_Relevant_Export_2022.xml']


In [10]:
import src.TaxAuthorityProvider.TaxAuthorities.Slovenia.TaxAuthorityProvider as tap
import src.TaxAuthorityProvider.TaxAuthorities.Slovenia.Schemas.ReportTypes as rt
import src.TaxAuthorityProvider.Schemas.Configuration as cf

### Dividends Reports

In [7]:
reportconfig = cf.TaxAuthorityConfiguration(fromDate=arrow.get("2022"), toDate=arrow.get("2023"))

dividendReport = sir.EDavkiDividendReport(reportConfig=reportconfig, taxPayerInfo=taxPayerInfo)
envelope = dividendReport.createReportEnvelope()

convertedCommonFormat = ibrkMapper.getGenericDividendLineFromIBRKCashTransactions(mergedReports.cashTransactions)

csvReport = dividendReport.generateDataFrameReport(convertedCommonFormat)
csvReport.to_csv(os.path.join(generatedDirectory, 'export.csv'), index=False)

xmlReport = dividendReport.generateXmlReport(convertedCommonFormat, envelope)


etree.ElementTree(xmlReport).write(os.path.join(generatedDirectory, "Doh_Div_3.xml"), xml_declaration=True, encoding='utf-8', pretty_print=True)



### Stock Trades

In [17]:
reportconfig = cf.TaxAuthorityConfiguration(fromDate=arrow.get("2022"), toDate=arrow.get("2023"))
convertedCommonFormat = ibkrProvider.transformBrokerEventsToBrokerAgnosticEvents(mergedReports)
groupingProcessor = stup.StagingToUnderlyingProcessor()


converted = groupingProcessor.generateGenericGroupings(convertedCommonFormat)

# print(convertedCommonFormat)
provider = tap.SlovenianTaxAuthorityProvider(taxPayerInfo=taxPayerInfo, reportConfig=reportconfig)
tradeReport = provider.generateExportForTaxAuthority(reportType=rt.SlovenianTaxAuthorityReportTypes.DOH_KDVP, data=converted)


tradeCsv = provider.generateSpreadsheetExport(reportType=rt.SlovenianTaxAuthorityReportTypes.DOH_KDVP, data=converted)
tradeCsv.to_csv(os.path.join(generatedDirectory, 'export-trades.csv'), index=False)



etree.ElementTree(tradeReport).write(os.path.join(generatedDirectory, "Doh_KDVP_9.xml"), xml_declaration=True, encoding='utf-8', pretty_print=True)


Failed processing stock lot (ID: 290300028, ISIN: US6742152076), found no match
Failed processing stock lot (ID: , ISIN: MHY8900D1085), found no match
Failed processing stock lot (ID: 259223542, ISIN: FR0014008WS3), found no match


### Derivatives Report

In [10]:
reportconfig = sir.EDavkiReportConfig(fromDate=arrow.get("2023"), toDate=arrow.get("2024"), ReportType=sir.EDavkiDocumentWorkflowType.ORIGINAL)
convertedCommonFormat = ibrkMapper.getGenericDerivativeTradeLinesFromIBRKTrades(mergedReports)

# print(convertedCommonFormat)
tradeReport = sir.EDavkiDerivativeReport(reportConfig=reportconfig, taxPayerInfo=taxPayerInfo)
envelope = tradeReport.createReportEnvelope()


test = tradeReport.convertTradesToIfiItems(convertedCommonFormat)





tradeCsv = tradeReport.generateDataFrameReport(convertedCommonFormat)
tradeCsv.to_csv(os.path.join(generatedDirectory, 'export-derivatives.csv'), index=False)



xmlReport = tradeReport.generateXmlReport(convertedCommonFormat, envelope)


etree.ElementTree(xmlReport).write(os.path.join(generatedDirectory, "D_IFI_4.xml"), xml_declaration=True, encoding='utf-8', pretty_print=True)


US0378331005
